In [ ]:
# 操作 browser 的 API
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 強制等待 (執行期間休息一下)
from time import sleep

# 執行 command 的時候用的
import os

# 子處理程序，用來取代 os.system 的功能
import subprocess

from bs4 import BeautifulSoup as bs
import requests as req

# 啟動瀏覽器工具的選項
my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")             #不開啟實體瀏覽器背景執行
my_options.add_argument("--start-maximized")        #最大化視窗
my_options.add_argument("--incognito")              #開啟無痕模式
my_options.add_argument("--disable-popup-blocking") #禁用彈出攔截
my_options.add_argument("--disable-notifications")  #取消 chrome 推播通知
my_options.add_argument("--lang=zh-TW")             #設定為正體中文

# 使用 Chrome 的 WebDriver
driver = webdriver.Chrome(
    options = my_options,
    service = Service(ChromeDriverManager().install())
)


# 建立一個iFoodie目錄
folderPath = 'iFoodie'
if not os.path.exists(folderPath):
    os.makedirs(folderPath)
    
def ifoodie_scraping(x:int,y:int):    
# 宣告總共的篇數    
    total=0 
    # 網頁範圍
    for page in range(x,y):
        url = f"https://ifoodie.tw/explore/%E5%8F%B0%E5%8C%97%E5%B8%82/list/%E5%A4%A7%E5%AE%89%E5%8D%80?page={page}" 
        res = req.get(url) 
        soup = bs(res.text, "lxml") 

        # 建立一個店家名稱list
        name = soup.select('div.jsx-3292609844.title>a.jsx-3292609844.title-text')
        listname=[i.get_text() for i in name]
      
        # 建立一個星評的空list
        rank= soup.select('div.jsx-1207467136.text')
        listrank=[i.get_text() for i in rank]

        # 建立一個地址的空list
        addr=soup.select('div.jsx-3292609844.address-row')
        listaddr=[i.get_text() for i in addr]

        # 建立一個連結的空list
        listlink=[]
        link = soup.select('div.jsx-3292609844.title>a.jsx-3292609844.title-text')
        listlink=["https://ifoodie.tw"+i['href'] for i in name]

        # 建立一個txt檔案
        with open(f"{folderPath}/大安區第{page}頁.txt", "w", encoding='utf-8') as file:

            # 宣告單頁的篇數
            count=0
            for i in range(1,15):

                # 資料寫進txt裡
                file.write('店名:'+listname[i]+'\n')
                file.write('評價等級:'+listrank[i]+'\n')
                file.write('地址:'+listaddr[i]+'\n')
                file.write('評論:'+'\n')

                # 篇數加1
                count+=1
                
                # 總篇數加1
                total+=1      
                print(f"目前第{page}頁第{count}篇:{listname[i]}執行中")

                # 在點選進入每個網頁
                driver.get(listlink[i])
                try:
                        # 等待"顯示更多"元素出現
                    WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located( 
                            (By.CSS_SELECTOR, "button.jss76.jss50.jss52.jss55.btn-more-checkin") 
                        )
                    )

                        #按下篩選元素，使項目浮現
                    driver.find_element(
                        By.CSS_SELECTOR, 
                        "button.jss76.jss50.jss52.jss55.btn-more-checkin"
                    ).click()

                    sleep(2)

                except TimeoutException:
                    print(f"第{page}頁-第{count}篇,評論偏少,沒等 到展開鍵元素")

                # 抓評論
                commen = driver.find_elements(
                    By.CSS_SELECTOR, 
                        'div.jsx-643342438.message>div.jsx-643342438'
                )

                print(f"第{page}頁第{count}篇:{listname[i]}已完成")

                # 評論寫入檔案中
                for obj in dates:
                    file.write(obj.get_attribute('innerText'))

                # 印出空白換行
                file.write('\n')
                file.write('\n')
                file.write('\n')
                file.write('\n')

        print(f'(第{page}頁完成')
        print(f"目前總共完成{total}個店家資料")
        

ifoodie_scraping()